In [41]:
#!pip install python-Levenshtein
import pandas as pd
import re
import numpy as np
from fuzzywuzzy import fuzz, process
import numpy as np
import pycountry as pyco

In [42]:
Attributes_DF=pd.read_csv('Extracted_Attributes.csv')

In [43]:
Attributes_DF.Possible_country_name[0].split('\n')

['conformed copy',
 'republic cote divoire',
 'reconstruction',
 'republic cote divoire',
 'reconstruction',
 'sodefor',
 'sodefor']

# Identifying Country from Cover Page
We do a fuzzy match between the Possible_country and the country names gotten from pycountry. 

In [44]:
def get_short_name(long_name):
    name=long_name.split(',')[0].lower()
    name=re.sub('peoples*|democr\w*|repub\w*','',name)
    re_string='[^0-9a-zA-Z\s.\(\)]+'
    name=re.sub(re_string, '',name)
    return name

In [45]:
Countries=pd.DataFrame({'Country_Code':[], 'Country_Name':[],'Official_name':[],'Short_name':[]})
for ind,country in enumerate(pyco.countries):
    name=country.name
    short_name=get_short_name(name)
    try: official=country.official_name
    except: official=name
    cut=name.split(',')[0]
    Countries.loc[ind]=[country.alpha_3,name,official,short_name]
#Is to easy to confuse mexico, so we add a new row
Countries.loc[ind+1]=['MEX','Mexico','United Mexican States','united mexican states']

In [46]:
Extended_DF=pd.DataFrame({'filename':[],'Possible_country':[]})
new_ind=0
for ind in Attributes_DF.index:
    if (ind%1000)==0:
        print(ind)
    filename=Attributes_DF.filename[ind]
    try:
        possible_countries=Attributes_DF.Possible_country_name[ind].split('\n')
        for possible in possible_countries:
            Extended_DF.loc[new_ind]=[filename,possible]
            new_ind+=1
    except:
        True

0
1000
2000
3000


In [47]:
Extended_DF.shape

(16074, 2)

In [48]:
#Combine the DataFrames
Extended_DF['Key']=1
Countries['Key']=1
Combined=Extended_DF.merge(Countries,on='Key',how='left')
Combined=Combined.drop('Key',axis=1)
Combined=Combined.drop('filename',axis=1).drop_duplicates()

In [49]:
def partial_match_long(x,y):
    return(fuzz.WRatio(x.lower(),y.lower()))
partial_match_long_vector = np.vectorize(partial_match_long)

def partial_match_short(x,y):
    return(fuzz.QRatio(x.lower(),y.lower()))
partial_match_short_vector = np.vectorize(partial_match_short)

In [50]:
score_treshold1=80
score_treshold2=50

Combined['Score1']=partial_match_long_vector(Combined['Possible_country'],Combined['Short_name'])
Combined['Max_Score1']=Combined.groupby(['Possible_country'])['Score1'].transform(max)
Combined=Combined[Combined.Score1>score_treshold1]
Combined=Combined[Combined.Score1==Combined.Max_Score1]

In [51]:
Combined['Score2']=partial_match_short_vector(Combined['Possible_country'],Combined['Short_name'])
Combined['Max_Score2']=Combined.groupby(['Possible_country'])['Score2'].transform(max)
Combined=Combined[Combined.Score2==Combined.Max_Score2]

In [52]:
Combined['Score3']=partial_match_short_vector(Combined['Possible_country'],Combined['Official_name'])
Combined['Max_Score3']=Combined.groupby(['Possible_country'])['Score3'].transform(max)
Combined=Combined[Combined.Score3==Combined.Max_Score3]

In [53]:
Combined['Score1'][Combined.Score1==0]=None
Combined['Score2'][Combined.Score2==0]=None
Combined['Score3'][Combined.Score3==0]=None
Combined['Score_final']=np.nanmean(Combined[['Score1','Score2','Score3']],axis=1)
Combined=Combined[Combined.Score_final>=score_treshold2]

In [54]:
#Recombine with the filenames
Combined=Combined[['Possible_country','Country_Code','Country_Name','Score_final','Score1']]

In [55]:
Identified_Countries=Extended_DF.merge(Combined,on='Possible_country').drop(['Key'],axis=1).drop_duplicates()

In [56]:
Identified_Countries.shape

(2936, 6)

In [57]:
Identified_Countries=Identified_Countries.sort_values('Score1',ascending=False).groupby('filename').first().reset_index()

In [58]:
Attributes_DF=Attributes_DF.merge(Identified_Countries,how='outer',on='filename')

In [59]:
Attributes_DF.Country_Code.isnull().sum()

604

In [86]:
Attributes_DF.rename({'Country_Code':'Country_Code_Cover'},axis=1,inplace=True)

# Extract Country from Address
We extract the country from address by finding which country or capital city from the world_bank_country_mappings data is in the address. We manually fix many common mistakes, like identifying "Niger" when the country is actually "Nigeria"

In [60]:
worldmap = pd.read_csv('world_bank_country_mappings.csv')

In [95]:
country_name = []
country_code = []
income_level = []
region = []
for i in Attributes_DF.index:
    address = Attributes_DF.Address[i]
    if type(address)==str:
        address = address.replace('\n','')
        address = address.replace(' ','')
        address = address.replace('  ','')
        address = address.replace("’","")
        address = address.lower()
        for j in range(len(worldmap['name'])):

            if worldmap['name'][j] == 'Africa':
                continue

            if worldmap['name'][j] == 'United States':
                continue

            if worldmap['name'][j] == 'World':
                continue

            if worldmap['name'][j] == 'Niger' and address.find('nigeria') > -1:
                continue

            if worldmap['name'][j] == 'Oman' and address.find('romania') > -1:
                continue

            if worldmap ['name'][j] == 'Oman' and address.find('phi') > -1:
                continue

            if worldmap ['name'][j] == 'Mali' and address.find('turkey') > -1:
                continue

            if worldmap['name'][j] == 'Guinea' and address.find('papua') > -1:
                countryname = 'Papua New Guinea'
                countrycode = 'PNG'
                incomelevel = 'Lower middle income'
                regi = 'East Asia & Pacific'
                break

            if worldmap['name'][j] == 'Spain' and address.find('trinidad')> -1:
                countryname = 'Trinidad and Tobago'
                countrycode = 'TTO'
                incomelevel = 'High income'
                regi = 'Latin America & Caribbean '
                break

            if worldmap['name'][j] == 'Chile' and address.find('paraguay')> -1:
                countryname = 'Paraguay'
                countrycode = 'PRY'
                incomelevel = 'Upper middle income'
                regi = 'Latin America & Caribbean '
                break

            else:    
                name = worldmap['name'][j]
                name = name.replace(' ','')
                name = name.replace("'","")
                name = name.replace('  ','')
                name = name.lower()
                cut = name.find(',')
                if cut > 0:
                    name = name[:cut]
                countryname = ''
                countrycode = ''
                incomelevel = ''
                regi = ''



                if address.find(name) > -1:
                    if name == 'congo' and address.find('brazzaville')>-1:
                        countryname = 'Congo,Rep.'
                        countrycode = 'COG'
                        incomelevel = 'Lower middle income'
                        regi = 'Sub-Saharan Africa'
                        break

                    else:  
                        countryname = worldmap['name'][j]
                        countrycode = worldmap['id'][j]
                        incomelevel = worldmap['incomeLevel.value'][j]
                        regi = worldmap['region.value'][j]
                        break            


                city = worldmap['capitalCity'][j]
                if type(city) == str:
                    city = city.replace(' ','')
                    city = city.replace("'","")
                    city = city.replace('  ','')
                    city = city.lower()
                    if address.find(name) == -1 and address.find(city) >-1:
                        countryname = worldmap['name'][j]
                        countrycode = worldmap['id'][j]
                        incomelevel = worldmap['incomeLevel.value'][j]
                        regi = worldmap['region.value'][j]
                        break                       


        if countryname == '':
            if address.find('méxico') > -1 or address.find('mexican') > -1:
                countryname = 'Mexico'
                countrycode = 'MEX'
                incomelevel = 'Upper middle income'
                regi = 'Latin America & Caribbean '

        if address.find('yugoslavia') > -1:
                countryname = 'Yugoslavia'
                countrycode = 'YUGOS'
                incomelevel = 'Upper middle income'
                regi = 'Europe & Central Asia'

        if countryname == '':
            countryname = None
            countrycode = None
            incomelevel = None
            regi = None
    else:
        countryname=countrycode=incomelevel=regi=None

    country_name.append(countryname)
    country_code.append(countrycode)
    income_level.append(incomelevel)
    region.append(regi)

In [96]:
Attributes_DF['Country_Code_Address'] = pd.Series(country_code)

# Compare Both Country Codes

In [98]:
cover_code = Attributes_DF['Country_Code_Cover']
address_code = Attributes_DF['Country_Code_Address']

In [99]:
same = []
nan = []
none = []
twonulls = []
twodiff = []


for i in range(3205):
    if cover_code[i] != address_code[i] and type(cover_code[i])== float:
        if address_code[i] == None:
            twonulls.append(i)
        else:
            nan.append(i)
    elif cover_code[i] != address_code[i] and type(cover_code[i])== str:
        if address_code[i] == None:
            none.append(i)
        else:
            twodiff.append(i)
    else:
        same.append(i)

In [100]:
print(len(same),len(nan),len(none),len(twonulls),len(twodiff))

2075 421 455 183 71


In [103]:
bettercode = []
for i in twodiff:
    address = Attributes_DF["Address"][i]
    address = address.replace('\n','')
    address = address.replace(' ','')
    address = address.replace('  ','')
    address = address.replace("’","")
    address = address.lower()
    
    if address_code[i] == 'BRA' and address.find('brazil')> -1:
        bettercode.append(i)
        continue
        
    if address_code[i] != 'YUGOS':
    
        ind = int(worldmap[worldmap['id'] == address_code[i]].index.values)
    
        country = worldmap['name'][ind]
        country = country.lower()
        
        if address.find(country) >-1:
            bettercode.append(i)
            continue
                
        city = worldmap['capitalCity'][ind]
    
        if type(city) == str:
            city = city.replace(' ','')
            city = city.replace("'","")
            city = city.replace('  ','')
            city = city.lower()
            if address.find(city) >-1:
                bettercode.append(i)

In [105]:
rest_twodiff = list(set(twodiff) - set(bettercode))

In [106]:
for i in rest_twodiff:
    if address_code[i] == 'Yugoslavia':
        continue
    if cover_code[i] == 'THA' or cover_code[i] == 'ALB':
        address_code[i] = cover_code[i]       

<ipython-input-106-c3713abad795>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_code[i] = cover_code[i]


In [107]:
for i in none:
    address_code[i] = cover_code[i]

<ipython-input-107-1f3121cab068>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_code[i] = cover_code[i]
